In [3]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.

In [4]:
pip install datasets

In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
import csv
import warnings
from typing import Any, Dict, Tuple
import re
from transformers import AutoTokenizer, pipeline


warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

# Dataset

In [ ]:
from datasets import load_dataset

#load the test data for the generation step for all language pairs
data = load_dataset("Eloquent/HalluciGen-Translation", "test_generation")


In [ ]:
data

In [6]:
#load model Llama3
model = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Prompt 2


*   Give model a role
*   Chain-of-Thought approach: examples to illustrate the task



In [ ]:
df_de_en = data["test_generation_de_en"].to_pandas()
df_de_en.head()

,id,langpair,source
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ..."


In [7]:
def generate_translation(source, target_language, example):
  answer_format = {"hyp+": "", "hyp-": ""}

  # prompt for translation generation task
  user_prompt = f'''
  As an AI model, your task is to generate two translation hypothesis given the 'src' below.
  The first translation labelled as 'hyp+' should be supported by 'src' and the second translation labelled as 'hyp-' should not be supported by 'src'.
  Provide the result in the following format: {answer_format}. Target language: {target_language}

  Here is an example to illustrate this:
  {example}

  Src: {source}

  Result:
  '''
  messages = [
      {"role": "system", "content": "You are a text generator. You are specialized in translating texts."},
      {"role": "user", "content": user_prompt},
    ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=250,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
  generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

  return generated_paraphrase

In [ ]:
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

In [ ]:
source = "My mom's food is better than my sister's."
result = generate_translation(source, "German", example)
extract_json_data(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



  As an AI model, your task is to generate two translation hypothesis given the 'src' below.
  The first translation labelled as 'hyp+' should be supported by 'src' and the second translation labelled as 'hyp-' should not be supported by 'src'.
  Provide the result in the following format: {'hyp+': '', 'hyp-': ''}. Target language: German

  Here is an example to illustrate this:
  src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
  hyp+: In the phase, wandering ants march at night and rest during the day. 
  This is a translation that is supported by the source. It is the exact translation of 'src'. 
  hyp-: In the nomadic phase, wandering ants march at night and rest during the day.
  This is a translation, which is not supported by the source. It includes an addition, which is "nomadic" in this case. In 'src', there is no mention about a 'nomadic' phase. Hence, it is labelled as 'hyp-'.

  Src: My mom's food is better than my sister's.

  Result:
  
{'

{'hyp+': 'Meine Mutter kocht besser als meine Schwester.',
 'hyp-': 'Meine Mutter kocht besser als meine Schwester in ihrer Küche.'}

## Generate dataset

In [ ]:

import os
import numpy as np
import pandas as pd

def save_results(target_language, df, csv_filename, example):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)
    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']

            try:
                # Generate paraphrase
                result = generate_translation(source, target_language, example)
                #print("source: ", source)
                extracted_hyp = extract_json_data(result)

                hyp_support = extracted_hyp['hyp+']
                #print("hyp+: ", hyp_support)
                hyp_hallu = extracted_hyp['hyp-']
                #print("hyp-: ", hyp_hallu)

                # Update DataFrame with new values
                chunk.at[index, 'hyp+'] = hyp_support
                chunk.at[index, 'hyp-'] = hyp_hallu
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
            print("finished chunk")
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)
            print("finished chunk")

        # Load CSV file
        df = pd.read_csv(csv_filename)
        display(df)
    return skipped_rows

# German-English

In [ ]:
df_de_en = data["test_generation_de_en"].to_pandas()
df_de_en.head()

,id,langpair,source
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ..."


In [ ]:
#load the trial data only for the German->English pair
trial_ds_de_en = load_dataset("Eloquent/HalluciGen-Translation", name="trial", split="trial_de_en")

In [ ]:
# load trial dataset to get one example for chain-of-thought approach
trial_de_en = trial_ds_de_en.to_pandas()

In [ ]:
trial_de_en.head()

,id,langpair,source,hyp1,hyp2,type,label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2


In [ ]:
# create example for user prompt
example = f'''src: {trial_de_en.iloc[0]['source']}
  hyp+: {trial_de_en.iloc[0]['hyp1']}
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: {trial_de_en.iloc[0]['hyp2']}
  This is a translation, which is not supported by the source. It includes an addition, which is "nomadic" in this case. In 'src', there is no mention about a 'nomadic' phase. Hence, it is labelled as 'hyp-'.'''
print(example)

src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
  hyp+: In the phase, wandering ants march at night and rest during the day. 
  This is a translation that is supported by the source. It is the exact translation of 'src'. 
  hyp-: In the nomadic phase, wandering ants march at night and rest during the day.
  This is a translation, which is not supported by the source. It includes an addition, which is "nomadic" in this case. In 'src', there is no mention about a 'nomadic' phase. Hence, it is labelled as 'hyp-'.


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_de_eng.csv"
skipped_rows = save_results("English", df_de_en, csv_filename, example)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, the offering of luxury guesthous...","In the countries, the offering of luxury guest..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in the UK, but Scottis...","This is common practice in the UK, but Scottis..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"Based on this fossil, it can be said that the ...","Based on this fossil, it can be said that the ..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never pass a car, build a well in our ...","We will never pass a car, build a well in our ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...","A minesweeper of the Avenger class, the ship w...","A minesweeper of the Avenger class, the ship w..."
5,5,de-en,Die Spitze war ein spezielles Heiligtum für de...,The tip was a special shrine for the god.,The peak was a sacred place for the god.
6,6,de-en,Kaltes Wetter ist vielleicht die einzige Gefah...,Cold weather is perhaps the only danger that t...,Cold weather is perhaps the only danger that t...
7,7,de-en,Das Fliegen in der Nähe eines Flughafens oder ...,Flying near an airport or over a crowd of peop...,Flying near an airport or over a crowd of peop...
8,8,de-en,"gab die US Navy an, sie sei dabei, den Zwisch...",The US Navy announced that it is investigating...,The US Navy announced that it is investigating...
9,9,de-en,In den letzten 20 Jahren ist die Auswahl in Up...,"In the last 20 years, the selection in Uptown ...","In the last 20 years, the exponential growth o..."


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 10 due to error in data extraction.


...

finished chunk


,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, the offering of luxury guesthous...","In the countries, the offering of luxury guest..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in the UK, but Scottis...","This is common practice in the UK, but Scottis..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"Based on this fossil, it can be said that the ...","Based on this fossil, it can be said that the ..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never pass a car, build a well in our ...","We will never pass a car, build a well in our ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...","A minesweeper of the Avenger class, the ship w...","A minesweeper of the Avenger class, the ship w..."
...,...,...,...,...,...
85,85,de-en,Wenn `` R '' die zugehörige Äquivalenzrelation...,NaN,NaN
86,86,de-en,"Es zeigt 362 verschiedene alte Holzarten, Büsc...","It shows 362 different old wood species, bushe...",The exhibit displays 362 different types of an...
87,87,de-en,"Obrovac ist eine Stadt in Norddalmatien, in Kr...","Obrovac is a town in Northern Dalmatia, in Cro...","Obrovac is a town in the north of Dalmatia, in..."
88,88,de-en,Danach wechselte er 1986 zu The Star und verli...,"After that, he switched to The Star in 1986 an...",He switched to The Star and left The Muslim.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, the offering of luxury guesthous...","In the countries, the offering of luxury guest..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in the UK, but Scottis...","This is common practice in the UK, but Scottis..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"Based on this fossil, it can be said that the ...","Based on this fossil, it can be said that the ..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never pass a car, build a well in our ...","We will never pass a car, build a well in our ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...","A minesweeper of the Avenger class, the ship w...","A minesweeper of the Avenger class, the ship w..."
...,...,...,...,...,...
95,95,de-en,Die Anzahl der gemeldeten Brände betrug Anfang...,The number of reported fires was 73 at the beg...,The number of reported fires was 73 at the beg...
96,96,de-en,Der Ponoru ist eine Nebenfluss des Horezu in R...,The Ponoru is a tributary of the Horezu in Rom...,The Ponoru is a main river of the Horezu in Ro...
97,97,de-en,Leslie Waddington eröffnete 1966 seine eigene ...,Leslie Waddington opened his own gallery in Co...,Leslie Waddington opened his own gallery in Co...
98,98,de-en,In der Zwischenzeit kaufte die Air Force die S...,"In the meantime, the Air Force bought the SR71...","In the meantime, the Air Force bought the SR71..."


In [ ]:
skipped_rows

[10, 26, 31, 41, 49, 52, 57, 60, 79, 84, 85]

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_de_eng.csv"
df_results = pd.read_csv(csv_filename)
len(df_results)

100

In [ ]:

def fix_skipped_rows(target_language, df, csv_filename, skipped_rows, example):
    skipped_rows2 = []
    for index in skipped_rows:
        row = df.loc[index]

        source = row['source']

        try:
            result = generate_translation(source, target_language,example)
            # Find the start and end indices of the JSON data
            start_index = result.find('{')
            end_index = result.find('}') + 1
            result_text = result[start_index:end_index]
            result_text = result_text.replace('"', "'").replace("{'hyp+': '", '{"hyp+": "').replace(".', 'hyp-': '", '.", "hyp-": "').replace(".',\n'hyp-': '", '.",\n"hyp-": "').replace(".', \n'hyp-': '", '.", \n"hyp-": "').replace("'}", '"}')
            print("result text ", result_text)
            #print("result text ", result_text)
            extracted_hyp = json.loads(result_text)
            df.at[index, 'hyp+'] =  extracted_hyp['hyp+']
            df.at[index, 'hyp-'] =  extracted_hyp['hyp-']
        except Exception as e:
            print(f"Skipping row {index} due to error in data extraction.")
            skipped_rows2.append(index)
            continue

    df.to_csv(csv_filename, index=False)
    return skipped_rows2


In [ ]:
skipped_rows = fix_skipped_rows("English", df_results, "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_de_eng_fixed.csv", skipped_rows, example)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Perry declared that he would'return to Texas to evaluate the result of tonight's primary and decide whether there is a further way for me in this race', but later said he would stay in the race and participate in the primaries in South Carolina on January 21.", "hyp-": "Perry declared that he would return to Texas to evaluate the result of tonight's primary and decide whether there is a further way for him in this race, but later said he would stay in the race and participate in the South Carolina primary on January 21, and then consider running for president."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "The worker dug a hole for the secretary.", "hyp-": "The worker dug a hole for the secretary's assistant."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

result text  {"hyp+": "The PBS show won more than two dozen Emmy Awards and its runtime is shorter than Sesame Street and Mister Rogers' Neighborhood.", "hyp-": "The PBS show won more than two dozen Emmy Awards and its runtime is much shorter than Sesame Street and Mister Rogers' Neighborhood."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Miranda quotes Voltaire: 'If we don't find something pleasant, we will find something new' and looks at Oscar with longing.", "hyp-": "Miranda quotes Voltaire: 'If we don't find something new, we will find something pleasant' and looks at Oscar with longing."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "The Swiss teacher Jürgen Reichen, who specialized in progressive education, founded this 'Writing to Read' method in 1982.", "hyp-": "The Swiss teacher Jürgen Reichen founded this 'Writing to Read' method in 1982, which is a part of progressive education."}
result text  {"hyp+": "When 'R' is the corresponding equivalence relation over every trivial component of 'U'.", "hyp-": "When 'R' is the corresponding equivalence relation over every trivial component of 'U' for a given group 'U'."}


In [ ]:
skipped_rows

[]

#English-German

In [ ]:
df_en_de = data["test_generation_en_de"].to_pandas()
df_en_de.head()

,id,langpair,source
0,0,en-de,He graduated from the College of Arts & Scienc...
1,1,en-de,"Most are independent nations, or associated wi..."
2,2,en-de,But there are a lot of things about birds that...
3,3,en-de,The police had trouble using their speed radar...
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t..."


### Example

In [ ]:
#load the trial data only for the German->English pair
trial_ds_en_de = load_dataset("Eloquent/HalluciGen-Translation", name="trial", split="trial_en_de")
trial_en_de = trial_ds_en_de.to_pandas()
trial_en_de.head()

,id,langpair,source,hyp1,hyp2,type,label
0,0,en-de,The days in the summer can lead to problems ge...,"Die Tage im Sommer können zu Problemen führen,...",Die sehr langen Tage im Sommer können zu Probl...,addition,hyp2
1,1,en-de,The vertical clearance under the bridge is 15 ...,Der vertikale Abstand unter der Brücke beträgt...,Der vertikale Abstand unter der Brücke beträgt...,date,hyp1
2,2,en-de,William Llewelyn Williams known as Llewelyn Wi...,"William Williams, bekannt als Llewelyn William...","Williajm Llewelyn Williams, bekannt als Llewel...",named-entity,hyp2
3,3,en-de,The single was released on Radio Airplay in It...,Die Single wurde am 12. Oktober 212 in Italien...,Die Single wurde am 12. Oktober 2012 in Italie...,number,hyp1
4,4,en-de,Water is spilling over the levee in a section ...,Wasser fließt über den Deich in einem Abschnit...,Wasser fließt über die Leibe in einem Abschnit...,natural,hyp2


In [ ]:
# create example for user prompt
example = f'''src: {trial_en_de.iloc[0]['source']}
  hyp+: {trial_en_de.iloc[0]['hyp1']}
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: {trial_en_de.iloc[0]['hyp2']}
  This is a translation, which is not supported by the source. It includes an addition, which is "sehr langen" in this case. In 'src', there is no mention about 'the very long' days in the summer. Hence, it is labelled as 'hyp-'.'''
print(example)

src: The days in the summer can lead to problems getting sufficient sleep and associated health issues.
  hyp+: Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme. 
  This is a translation that is supported by the source. It is the exact translation of 'src'. 
  hyp-: Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.
  This is a translation, which is not supported by the source. It includes an addition, which is "sehr langen" in this case. In 'src', there is no mention about 'the very long' days in the summer. Hence, it is labelled as 'hyp-'.


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_de.csv"
skipped_rows = save_results("German", df_en_de, csv_filename,example)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 5 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er graduierte an der College of Arts & Science...,Er graduierte an der renommierten College of A...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die wie e...","Aber es gibt viele Dinge über Vögel, die sehr ..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei hatte Schwierigkeiten, ihre Geschw..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","Der Giza-Plateau, auch Giza-Nekropole in der ä...","Der Giza-Plateau, auch Giza-Nekropole in der ä..."
5,5,en-de,Prime Minister Stephen Harper has agreed to se...,NaN,NaN
6,6,en-de,The temple of Artemis was destroyed on July 21...,Der Tempel von Artemis wurde am 21. Juli 356 v...,Der Tempel von Artemis wurde am 21. Juli 356 v...
7,7,en-de,South Africa have defeated the All Blacks (New...,Südafrika hat die All Blacks (Neuseeland) in e...,Südafrika hat die All Blacks (Neuseeland) in e...
8,8,en-de,"condition is stable, though he will be isolat...","Die Bedingung ist stabil, obwohl er zu Hause f...","Die Bedingung ist stabil, obwohl er zu Hause f..."
9,9,en-de,"The world has over 5,000 different languages, ...",Die Welt hat mehr als 5.000 verschiedene Sprac...,"Die Welt hat über 5.000 verschiedene Sprachen,..."


...

Skipping row 89 due to error in data extraction.
finished chunk


,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er graduierte an der College of Arts & Science...,Er graduierte an der renommierten College of A...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die wie e...","Aber es gibt viele Dinge über Vögel, die sehr ..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei hatte Schwierigkeiten, ihre Geschw..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","Der Giza-Plateau, auch Giza-Nekropole in der ä...","Der Giza-Plateau, auch Giza-Nekropole in der ä..."
...,...,...,...,...,...
85,85,en-de,The NSA has its own internal data format that ...,"Die NSA hat ihr eigenes internes Datenformat, ...","Die NSA hat ein eigenes internes Datenformat, ..."
86,86,en-de,"It was in the early days of GPS, the car knew ...","Es war in den frühen Tagen von GPS, das Auto w...","Es war in den ersten Jahren von GPS, das Auto ..."
87,87,en-de,He could ask that because he knew New Yorkers ...,"Er könnte das fragen, weil er New Yorker kannt...","Er könnte das fragen, weil er sehr viele New Y..."
88,88,en-de,"Information is powerful, but it is how we use ...","Die Informationen sind mächtig, aber es ist, w...","Die Informationen sind mächtig, aber es ist, w..."


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er graduierte an der College of Arts & Science...,Er graduierte an der renommierten College of A...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die wie e...","Aber es gibt viele Dinge über Vögel, die sehr ..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei hatte Schwierigkeiten, ihre Geschw..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","Der Giza-Plateau, auch Giza-Nekropole in der ä...","Der Giza-Plateau, auch Giza-Nekropole in der ä..."
...,...,...,...,...,...
95,95,en-de,"If you invent something, Tesla invented electr...","Wenn du etwas erfunden hast, hat Tesla elektri...","Wenn du etwas erfunden hast, hat Tesla elektri..."
96,96,en-de,"LP: The amazing thing about this is this is, I...","Das bemerkenswerte an diesem ist, dass es, ich...","Das bemerkenswerte an diesem ist, dass es, ich..."
97,97,en-de,"LP: The amazing thing about this is this is, I...","Das erstaunliche an diesem ist, dass es, ich m...","Das erstaunliche an diesem ist, dass es, ich m..."
98,98,en-de,"""She said, """"I used to have a job but I had to...","Sie sagte, ‚Ich hatte einmal einen Job, aber i...","Sie sagte, ‚Ich hatte einmal einen Job, aber i..."


In [ ]:
skipped_rows

[5, 21, 33, 45, 57, 62, 89]

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_de.csv"

df_results = pd.read_csv(csv_filename)
df_results.head()

,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er graduierte an der College of Arts & Science...,Er graduierte an der renommierten College of A...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die wie e...","Aber es gibt viele Dinge über Vögel, die sehr ..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei hatte Schwierigkeiten, ihre Geschw..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","Der Giza-Plateau, auch Giza-Nekropole in der ä...","Der Giza-Plateau, auch Giza-Nekropole in der ä..."


In [ ]:
skipped_rows = fix_skipped_rows("German", df_results, "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_de_fixed.csv", skipped_rows, example)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Der kanadische Premierminister Stephen Harper hat sich geeignet, das Regierungsgesetz 'Clean Act' an ein parteiübergreifendes Gremium zur Überprüfung zu senden, bevor seine zweite Lesung, nach dem 25-minütigen Treffen mit dem NDP-Führer Jack Layton im PMO am Dienstag.", "hyp-": "Der kanadische Premierminister Stephen Harper hat sich geeignet, das Regierungsgesetz 'Clean Act' an ein parteiübergreifendes Gremium zur Überprüfung zu senden, bevor seine zweite Lesung, nach dem 25-minütigen Treffen mit dem NDP-Führer Jack Layton im PMO am Dienstag, nach einem langen Gespräch."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Am 15. August 1940 invasionierten die Alliierten das südliche Frankreich, die Invasion wurde als ''Operation Dragoon'' bezeichnet.", "hyp-": "Am 15. August 1940 invasionierten die Alliierten das gesamte Frankreich, die Invasion wurde als ''Operation Dragoon'' bezeichnet."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

result text  {"hyp+": "Die Art wurde erstmals formell von dem Botaniker Stephan Endlicher im Jahr 1846 beschrieben, als Teil der Arbeit 'Irideae Plantae Preissianae' von Johann Georg Christian Lehmann.", "hyp-": "Die Art wurde erstmals formell von dem Botaniker Stephan Endlicher im Jahr 1846 beschrieben, als Teil einer umfangreichen Studie über die Blumen von Südamerika, die von Johann Georg Christian Lehmann und Stephan Endlicher gemeinsam verfasst wurde."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Für Australier ist die Vorstellung von 'flauem Weiß' Kaffee fremd. Ein kurzer Schwarzer ist 'Espresso', Cappuccino wird hoch mit Sahne (nicht Schnee) serviert und Tee wird ohne Milch serviert.", "hyp-": "Für Australier ist die Vorstellung von sehr starkem 'flauem Weiß' Kaffee fremd. Ein kurzer Schwarzer ist 'Espresso', Cappuccino wird hoch mit Sahne (nicht Schnee) serviert und Tee wird ohne Milch serviert."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Obwohl es sehr verbreitet ist, insbesondere unter Nicht-Romani, wird das Wort 'Zigeuner' oft als peinlich empfunden, weil es negative Stereotype und ungenaue Vorstellungen von Menschen mit Romani-Wurzeln verbindet.", "hyp-": "Obwohl es sehr verbreitet ist, insbesondere unter Nicht-Romani, wird das Wort 'Zigeuner' oft als peinlich empfunden, weil es negative Stereotype und ungenaue Vorstellungen von Menschen mit Romani-Wurzeln verbindet und außerdem ihre Kultur nicht versteht."}
result text  {"hyp+": "Und schließlich, nach vier vollständigen Stunden meines Liegens und Starrens darauf, läutete das Telefon, und ich schaffte es irgendwie, es aufzunehmen, und es war mein Vater, und ich sagte:,,Ich stecke in ernster Schwierigkeit. Wir müssen etwas tun.'', 'hyp-': 'Und schließlich, nach vier vollständigen Stunden meines Liegens und Starrens darauf, läutete das Telefon, und ich schaffte es irgendwie, es aufzunehmen, und es war mein Vater, und ich sagte:,,Ich stecke in se

# English-French

In [ ]:
df_en_fr = data["test_generation_en_fr"].to_pandas()
df_en_fr.head()

,id,langpair,source
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...
2,2,en-fr,"Only the effects of catastrophizing, not depre..."
3,3,en-fr,It can be customized to make it easy to read a...
4,4,en-fr,Stars give off light and heat because of the e...


### Example

In [ ]:
#load the trial data only for the German->English pair
trial_ds_en_fr = load_dataset("Eloquent/HalluciGen-Translation", name="trial", split="trial_en_fr")
trial_en_fr = trial_ds_en_fr.to_pandas()
trial_en_fr.head()

,id,langpair,source,hyp1,hyp2,type,label
0,0,en-fr,The Madhya Police recovered the stolen laptop ...,La police de Madhya Pradesh a récupéré l'ordin...,La police de Madhya a récupéré l'ordinateur po...,addition,hyp1
1,1,en-fr,The vote must still be ratified by the full IO...,Le vote doit encore être ratifié par le CIO lo...,Le vote doit encore être ratifié par le CIO lo...,date,hyp2
2,2,en-fr,The Mumbai attackers arrived via boat on Novem...,Les assaillants de Mumbai sont arrivés par bat...,Les assaillants de Mumbai sont arrivés par bat...,date,hyp1
3,3,en-fr,Among the prominent women of Suriname are Jenn...,Parmi les femmes éminentes du Suriname figuren...,"Jennifer Simons, Marijke Djwalapersad, Elisabe...",named-entity,hyp2
4,4,en-fr,Taylor remained active until his death as a sc...,Taylopr est resté actif dans le baseball en ta...,Taylor est resté actif dans le baseball en tan...,named-entity,hyp1


In [ ]:
# create example for user prompt
example = f'''src: {trial_en_fr.iloc[0]['source']}
  hyp+: {trial_en_fr.iloc[0]['hyp2']}
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: {trial_en_fr.iloc[0]['hyp1']}
  This is a translation, which is not supported by the source. It includes an addition, which is 'Pradesh' in this case. In 'src', there is no mention about 'Pradesh'. Hence, it is labelled as 'hyp-'.'''
print(example)

src: The Madhya Police recovered the stolen laptop and mobile phone.
  hyp+: La police de Madhya a récupéré l'ordinateur portable et le téléphone portable volés.
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: La police de Madhya Pradesh a récupéré l'ordinateur portable et le téléphone portable volés.
  This is a translation, which is not supported by the source. It includes an addition, which is 'Pradesh' in this case. In 'src', there is no mention about 'Pradesh'. Hence, it is labelled as 'hyp-'.


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_fr.csv"
skipped_rows = save_results("French", df_en_fr, csv_filename, example)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 1 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 3 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 90 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 99 due to error in data extraction.
finished chunk


,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né à la capitale croate,, Bobek est devenu cél...","Né à la capitale croate, Zagreb, Bobek est dev..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...,NaN,NaN
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Seuls les effets du catastrophisme, et non la ...","Seuls les effets du catastrophisme, et non la ..."
3,3,en-fr,It can be customized to make it easy to read a...,NaN,NaN
4,4,en-fr,Stars give off light and heat because of the e...,NaN,NaN
...,...,...,...,...,...
95,95,en-fr,"William Llewelyn Williams, better known as Lle...","William Llewelyn Williams, mieux connu sous le...","William Llewelyn Williams, mieux connu sous le..."
96,96,en-fr,"Flavia Alejandra Gleske Fajin, better known as...","Flavia Alejandra Gleske Fajin, mieux connue so...","Flavia Alejandra Gleske Fajin, mieux connue so..."
97,97,en-fr,"In the 1930s and 1940s, he was elected four ti...","Dans les années 1930 et 1940, il a été élu qua...","Dans les années 1930 et 1940, il a été élu sep..."
98,98,en-fr,In 1924 he was an invited spokesperson for the...,"En 1924, il a été un porte-parole invité pour ...","En 1924, il a été un porte-parole invité pour ..."


In [ ]:
skipped_rows

[1,
 3,
 4,
 6,
 8,
 9,
 10,
 11,
 13,
 14,
 17,
 18,
 19,
 21,
 24,
 26,
 27,
 29,
 30,
 31,
 32,
 34,
 35,
 37,
 38,
 44,
 45,
 49,
 51,
 52,
 53,
 55,
 58,
 59,
 60,
 61,
 62,
 63,
 65,
 69,
 70,
 72,
 78,
 80,
 82,
 86,
 87,
 90,
 99]

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_fr.csv"

df_results = pd.read_csv(csv_filename)
df_results.head()

,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né à la capitale croate,, Bobek est devenu cél...","Né à la capitale croate, Zagreb, Bobek est dev..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...,NaN,NaN
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Seuls les effets du catastrophisme, et non la ...","Seuls les effets du catastrophisme, et non la ..."
3,3,en-fr,It can be customized to make it easy to read a...,NaN,NaN
4,4,en-fr,Stars give off light and heat because of the e...,NaN,NaN


In [ ]:
skipped_rows = fix_skipped_rows("French", df_results, "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_fr_fixed.csv", skipped_rows, example )

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Utiliser des applications de navigation GPS peut être la méthode la plus facile et la plus pratique pour se diriger lorsque vous êtes hors de votre pays d'origine.", "hyp-": "Utiliser des applications de navigation GPS peut être la méthode la plus facile et la plus pratique pour se diriger dans un pays étranger."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Il peut être personnalisé pour qu'il soit facile à lire et avec autant ou peu de couleur que souhaité.", "hyp-": "Il peut être personnalisé pour qu'il soit facile à lire et avec autant ou peu de couleur que souhaité, ainsi que pour être lu par les personnes aveugles."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

result text  {"hyp+": "La dernière ligne du troisième verse a été modifiée pendant le règne d'Alexandre Ier de Yougoslavie dans 'Kralja Aleksandra, Bože hrani ''. ', 'hyp-': 'La dernière ligne du troisième verse a été modifiée pendant le règne d'Alexandre Ier de Serbie en 'Kralja Aleksandra, Bože hrani ''. "}
Skipping row 86 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Le 4 avril 1892, le deuxième navire, 'Missouri', a livré 2 500 tonnes de grains et de fleurs de maïs à Liepaja.", "hyp-": "Le 4 avril 1892, le deuxième navire, 'Missouri', a livré 2 500 tonnes de grains, de maïs et de fleurs à Liepaja."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Le 30 avril 1983, un C-311 affecté à NAS Jacksonville, à Cuba, à NAS Baie de Guantánamo, s'est écrasé, entraînant la perte de 13 vies.", "hyp-": "Le 30 avril 1983, un C-311 affecté à NAS Jacksonville, à Cuba, à NAS Baie de Guantánamo, à NAS Key West, s'est écrasé, entraînant la perte de 13 vies."}
result text  {"hyp+": "Simyo appartient au groupe néerlandais de télécommunications KPN, après acquisition du reste d'E-Plus le 14 mars.", "hyp-": "Simyo appartient au groupe néerlandais de télécommunications KPN, après acquisition du reste d'E-Plus le 14 mars 2023."}


# French-English

In [ ]:
df_fr_en = data["test_generation_fr_en"].to_pandas()
df_fr_en.head()

,id,langpair,source
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...
1,1,fr-en,"est relativement facile en italien, car la pl..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...
3,3,fr-en,Northern avait sollicité une aide en raison de...
4,4,fr-en,"Au fil du temps, à mesure que la population s'..."


### Example

In [ ]:
#load the trial data only for the German->English pair
trial_ds_fr_en = load_dataset("Eloquent/HalluciGen-Translation", name="trial", split="trial_fr_en")
trial_fr_en = trial_ds_fr_en.to_pandas()
trial_fr_en.head()

,id,langpair,source,hyp1,hyp2,type,label
0,0,fr-en,"Mélangez les poudres sèches ensemble, puis, av...","Mix the two dried powders together, then, with...","Mix the dried powders together, then, with cle...",addition,hyp1
1,1,fr-en,Le vote doit encore être ratifié par l'ensembl...,The agreement will still have to be ratified b...,The agreement will still have to be ratified b...,date,hyp2
2,2,fr-en,Le tailleur a appelé la déménageuse.,The tailor called the female mover.,The tailor called the male mover.,gender,hyp2
3,3,fr-en,"Dans les années 1930 et 1940, il fut élu à qua...","In the 1930s and 1940s, he was elected four ti...","Massachusetts voted for Al Smith in 1928, for ...",named-entity,hyp1
4,4,fr-en,"La 99e a soutenu l'opération Urgent Wury, qui ...","Shortly after its activation, the 99th support...","The 919th supported the Urgent Wury operation,...",number,hyp2


In [ ]:
# create example for user prompt
example = f'''src: {trial_fr_en.iloc[1]['source']}
  hyp+: {trial_fr_en.iloc[1]['hyp1']}
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: {trial_fr_en.iloc[1]['hyp2']}
  This is a translation, which is not supported by the source. The date is wrong. The translation contains "May", instead of "October". Hence, it is labelled as 'hyp-'.'''
print(example)

src: Le vote doit encore être ratifié par l'ensemble du CIO lors de sa réunion d'octobre à Copenhague.
  hyp+: The agreement will still have to be ratified by the IOC at its Oct. meeting in Copenhagen.
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: The agreement will still have to be ratified by the IOC at its May meeting in Copenhagen.
  This is a translation, which is not supported by the source. The date is wrong. The translation contains "May", instead of "October". Hence, it is labelled as 'hyp-'.


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_fr_en.csv"
skipped_rows = save_results("English", df_fr_en, csv_filename, example)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly inaugurated a new Electoral Council (...,Martelly inaugurated a new Electoral Council (...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy in Italian, because most...","It is relatively difficult in Italian, because..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern had sought help due to its exposure t...,Northern had sought help due to its exposure t...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","Over time, as the population adapts to its new...","Over time, as the population adapts to its new..."
5,5,fr-en,L'archipel est un État en libre association av...,The archipelago is a state in free association...,The archipelago is a state in free association...
6,6,fr-en,"Bien que Goma soit raisonnablement sûre, toute...","Although Goma is reasonably safe, any visit mu...","Although Goma is reasonably safe, any visit mu..."
7,7,fr-en,Donnez également des exemplaires de la régleme...,Also provide copies of the regulations and con...,Also provide copies of the regulations and con...
8,8,fr-en,Parmi les sports populaires figurent le footba...,"Among popular sports are football, basketball,...","Among popular sports are football, basketball,..."
9,9,fr-en,"La société de gestion du groupe, HK Management...","The management company of the group, HK Manage...","The management company of the group, HK Manage..."


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 11 due to error in data extraction.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 90 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly inaugurated a new Electoral Council (...,Martelly inaugurated a new Electoral Council (...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy in Italian, because most...","It is relatively difficult in Italian, because..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern had sought help due to its exposure t...,Northern had sought help due to its exposure t...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","Over time, as the population adapts to its new...","Over time, as the population adapts to its new..."
...,...,...,...,...,...
95,95,fr-en,"La couleur de capuchon varie, de marron à jaun...","The capuchon color varies, from brown to yello...","The capuchon color varies, from blue to green,..."
96,96,fr-en,La rivière Arieşul Mare est un affluent de la ...,The Arieșul Mare River is a tributary of the V...,The Arieșul Mare River is a tributary of the V...
97,97,fr-en,"Born est né en Allemagne de parents juifs, le ...","Born was born in Germany to Jewish parents, th...","Born was born in France to Jewish parents, the..."
98,98,fr-en,"À proximité se trouve la rivière Wallowa, un a...","Nearby is the Wallowa River, a tributary of th...","Nearby is the Wallowa River, a tributary of th..."


In [ ]:
skipped_rows

[11, 35, 36, 42, 48, 54, 71, 74, 75, 82, 83, 86, 88, 89, 90]

In [ ]:
# fix skipped rows
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_fr_en.csv")

In [ ]:
df.head()

,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly inaugurated a new Electoral Council (...,Martelly inaugurated a new Electoral Council (...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy in Italian, because most...","It is relatively difficult in Italian, because..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern had sought help due to its exposure t...,Northern had sought help due to its exposure t...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","Over time, as the population adapts to its new...","Over time, as the population adapts to its new..."


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_fr_en_fixed.csv"
skipped_rows = fix_skipped_rows("English", df, csv_filename, skipped_rows, example)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "The problem started on January 1st when dozens of local residents began complaining to the Obanazawa post office that they had not received their traditional and regular New Year's cards.", "hyp-": "The problem started on December 31st when dozens of local residents began complaining to the Obanazawa post office that they had not received their traditional and regular Christmas cards."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "In 2012, Gil joined the television program - Remake of the Salvador - Royal Films 'Mundo Man ay Magunaw' in the role of Jennifer Pena.", "hyp-": "In 2018, Gil joined the television program - Remake of the Salvador - Royal Films 'Mundo Man ay Magunaw' in the role of Jennifer Pena."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

result text  {"hyp+": "Her admiral rejected the request until it was too late and the 'Conte di Cavour' had to use a deeper sandbank on November 12 at 04:30.", "hyp-": "Her admiral rejected the request until it was too late and the 'Conte di Cavour' had to use a deeper sandbank on December 12 at 04:30."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "His name, 'Afolabi', means 'Born into wealth'. His nickname in Nigeria is Robocop due to his stiff movements.", "hyp-": "His name, 'Afolabi', means 'Born into poverty'. His nickname in Nigeria is Superman due to his athletic movements."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Daniil Ostrogski also states that Dmytro is the father of Bilinsky, also known as 'Danylo Dmytrovych'', 'hyp-': 'Daniil Ostrogski also states that Dmytro is the son of Bilinsky, also known as 'Danylo Dmytrovych'"}
Skipping row 89 due to error in data extraction.
result text  {"hyp+": "This requires continuous monitoring of real partial pressures over time, and for maximum efficiency, real-time processing by the diver's computer decompression unit.", "hyp-": "This requires continuous monitoring of real partial pressures over time, and for maximum efficiency, real-time processing by the doctor's computer decompression unit."}


In [ ]:
skipped_rows

[75, 89]

## Find skipped rows

In [1]:

def find_skipped_rows(df):
  r, _ = np.where(df.isna())
  u, indices = np.unique(r, return_index=True)
  return u



In [8]:
## English-german
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_de_fixed.csv")
u = find_skipped_rows(df)
print(u)


[89]


In [9]:
example = f''' src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
  hyp+: In the phase, wandering ants march at night and rest during the day.
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: In the nomadic phase, wandering ants march at night and rest during the day.
  This is a translation, which is not supported by the source. It includes an addition, which is "nomadic" in this case. In 'src', there is no mention about a 'nomadic' phase. Hence, it is labelled as 'hyp-'.'''

print(example)

 src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
  hyp+: In the phase, wandering ants march at night and rest during the day. 
  This is a translation that is supported by the source. It is the exact translation of 'src'. 
  hyp-: In the nomadic phase, wandering ants march at night and rest during the day.
  This is a translation, which is not supported by the source. It includes an addition, which is "nomadic" in this case. In 'src', there is no mention about a 'nomadic' phase. Hence, it is labelled as 'hyp-'.


In [10]:
df.iloc[89]["source"]

'"And finally, after four full hours of my lying and staring at it, the phone rang, and somehow I managed to pick it up, and it was my father, and I said, ""I\'m in serious trouble. We need to do something."""'

In [12]:
translation = generate_translation(df.iloc[89]["source"], "German",example)
print(translation)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'hyp+': "Und schließlich, nach vier vollen Stunden meines Liegens und Starrens darauf, ging das Telefon, und ich gelang es irgendwie, es abzunehmen, und es war mein Vater, und ich sagte,,,Ich stecke in großem Ärger. Wir müssen etwas tun.""", 'hyp-': "Und schließlich, nach vier vollen Stunden meines Liegens und Starrens darauf, ging das Telefon, und ich gelang es irgendwie, es abzunehmen, und es war mein Vater, und ich sagte,,,Ich stecke in großem Ärger. Wir müssen etwas tun, um unser Verhältnis zu verbessern."""}


In [13]:
## German-English
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_de_eng_fixed.csv")
u = find_skipped_rows(df)
print(u)


[]


In [14]:
## French-English
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_fr_en_fixed.csv")
u = find_skipped_rows(df)
print(u)


[75 89]


In [15]:
translation = generate_translation(df.iloc[75]["source"], "English",example)
print(translation)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'hyp+': 'Quentin Collins, Ed Griffin, in "House of Despair", say that the inhabitants of Collinsport have not forgotten how "the winter girl disappeared inexplicably."', 'hyp-': 'Quentin Collins, Ed Griffin, in "House of Despair", say that the inhabitants of Collinsport have not forgotten how "the winter girl was murdered inexplicably".'}


In [16]:
translation = generate_translation(df.iloc[89]["source"], "English",example)
print(translation)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'hyp+': "Daniil Ostrogski also states that Dmytro is the father of Bilinsky, also known as 'Danylo Dmytrovych'.", 'hyp-': "Daniil Ostrogski declares that Dmytro is the father of Bilinsky, also known as 'Danylo Dmytrovych', and this information is widely recognized."}


In [18]:
## English-French
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_fr_fixed.csv")
u = find_skipped_rows(df)
print(u)


[86]


In [22]:
example = f'''src: The Madhya Police recovered the stolen laptop and mobile phone.
  hyp+: La police de Madhya a récupéré l'ordinateur portable et le téléphone portable volés.
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: La police de Madhya Pradesh a récupéré l'ordinateur portable et le téléphone portable volés.
  This is a translation, which is not supported by the source. It includes an addition, which is 'Pradesh' in this case. In 'src', there is no mention about 'Pradesh'. Hence, it is labelled as 'hyp-'.'''
print(example)

src: The Madhya Police recovered the stolen laptop and mobile phone.
  hyp+: La police de Madhya a récupéré l'ordinateur portable et le téléphone portable volés.
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: La police de Madhya Pradesh a récupéré l'ordinateur portable et le téléphone portable volés.
  This is a translation, which is not supported by the source. It includes an addition, which is 'Pradesh' in this case. In 'src', there is no mention about 'Pradesh'. Hence, it is labelled as 'hyp-'.


In [23]:
translation = generate_translation(df.iloc[86]["source"], "French",example)
print(translation)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'hyp+': 'La dernière ligne de la troisième strophe a été modifiée pendant le règne d'Alexandre Ier de Yougoslavie dans "Kralja Aleksandra, Bože hrani".', 'hyp-': 'La dernière ligne de la troisième strophe a été modifiée pendant le règne d'Alexandre Ier de Serbie en "Kralja Aleksandra, Bože hrani".'}


## Remove extra columns

In [43]:
# English-German
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_de_fixed2.csv", sep=';')
df.head()

,id,langpair,source,hyp+,hyp-,Unnamed: 5
0,0,en-de,He graduated from the College of Arts & Scienc...,Er graduierte an der College of Arts & Science...,Er graduierte an der renommierten College of A...,NaN
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...,NaN
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die wie e...","Aber es gibt viele Dinge über Vögel, die sehr ...",NaN
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei hatte Schwierigkeiten, ihre Geschw...",NaN
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","Der Giza-Plateau, auch Giza-Nekropole in der ä...","Der Giza-Plateau, auch Giza-Nekropole in der ä...",NaN


In [45]:
df = df.drop(['langpair', 'source'], axis=1)
df.head()

,id,hyp+,hyp-,Unnamed: 5
0,0,Er graduierte an der College of Arts & Science...,Er graduierte an der renommierten College of A...,NaN
1,1,Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...,NaN
2,2,"Aber es gibt viele Dinge über Vögel, die wie e...","Aber es gibt viele Dinge über Vögel, die sehr ...",NaN
3,3,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei hatte Schwierigkeiten, ihre Geschw...",NaN
4,4,"Der Giza-Plateau, auch Giza-Nekropole in der ä...","Der Giza-Plateau, auch Giza-Nekropole in der ä...",NaN


In [46]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_de_final.csv", mode='w', header=True, index=False)


In [50]:
## German-English
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_de_eng_fixed.csv")
df.head()


,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, the offering of luxury guesthous...","In the countries, the offering of luxury guest..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in the UK, but Scottis...","This is common practice in the UK, but Scottis..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"Based on this fossil, it can be said that the ...","Based on this fossil, it can be said that the ..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never pass a car, build a well in our ...","We will never pass a car, build a well in our ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...","A minesweeper of the Avenger class, the ship w...","A minesweeper of the Avenger class, the ship w..."


In [51]:
df = df.drop(['langpair', 'source'], axis=1)
df.head()

,id,hyp+,hyp-
0,0,"In countries, the offering of luxury guesthous...","In the countries, the offering of luxury guest..."
1,1,"This is common practice in the UK, but Scottis...","This is common practice in the UK, but Scottis..."
2,2,"Based on this fossil, it can be said that the ...","Based on this fossil, it can be said that the ..."
3,3,"We will never pass a car, build a well in our ...","We will never pass a car, build a well in our ..."
4,4,"A minesweeper of the Avenger class, the ship w...","A minesweeper of the Avenger class, the ship w..."


In [52]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_de_en_final.csv", mode='w', header=True, index=False)

In [54]:
## French-English
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_fr_en_fixed2.csv", sep=';')
df.head()

,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly inaugurated a new Electoral Council (...,Martelly inaugurated a new Electoral Council (...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy in Italian, because most...","It is relatively difficult in Italian, because..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern had sought help due to its exposure t...,Northern had sought help due to its exposure t...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","Over time, as the population adapts to its new...","Over time, as the population adapts to its new..."


In [55]:
df = df.drop(['langpair', 'source'], axis=1)
df.head()

,id,hyp+,hyp-
0,0,Martelly inaugurated a new Electoral Council (...,Martelly inaugurated a new Electoral Council (...
1,1,"It is relatively easy in Italian, because most...","It is relatively difficult in Italian, because..."
2,2,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,Northern had sought help due to its exposure t...,Northern had sought help due to its exposure t...
4,4,"Over time, as the population adapts to its new...","Over time, as the population adapts to its new..."


In [56]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_fr_en_final.csv", mode='w', header=True, index=False)


In [59]:
## English-French
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_fr_fixed2.csv", sep=';')

In [60]:
df.head()

,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né à la capitale croate,, Bobek est devenu cél...","Né à la capitale croate, Zagreb, Bobek est dev..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...,Utiliser des applications de navigation GPS pe...,Utiliser des applications de navigation GPS pe...
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Seuls les effets du catastrophisme, et non la ...","Seuls les effets du catastrophisme, et non la ..."
3,3,en-fr,It can be customized to make it easy to read a...,Il peut être personnalisé pour qu'il soit faci...,Il peut être personnalisé pour qu'il soit faci...
4,4,en-fr,Stars give off light and heat because of the e...,Les étoiles émettent de la lumière et de la ch...,Les étoiles émettent de la lumière et de la ch...


In [61]:
df = df.drop(['langpair', 'source'], axis=1)
df.head()

,id,hyp+,hyp-
0,0,"Né à la capitale croate,, Bobek est devenu cél...","Né à la capitale croate, Zagreb, Bobek est dev..."
1,1,Utiliser des applications de navigation GPS pe...,Utiliser des applications de navigation GPS pe...
2,2,"Seuls les effets du catastrophisme, et non la ...","Seuls les effets du catastrophisme, et non la ..."
3,3,Il peut être personnalisé pour qu'il soit faci...,Il peut être personnalisé pour qu'il soit faci...
4,4,Les étoiles émettent de la lumière et de la ch...,Les étoiles émettent de la lumière et de la ch...


In [62]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt2/results_llama3_en_fr_final.csv", mode='w', header=True, index=False)